In [2]:
import pandas as pd
import numpy as np


#  Olympic data set analysis

This analysis explores the relationship between a country's economic status and its Olympic performance, focusing on whether wealthier nations perform better in the Games and whether hosting the Olympics boosts the economy. The data, spanning from the 2000 to 2023 Olympic Games, will include medal counts and GDP figures. The ETL process will transform raw CSV files into fact and dimension tables, which will be organized in a data warehouse using PGAdmin. Atoti will be used for visualizations to answer these key questions.

### Main questions we aim to answer:

1. Do Countries with a higher gdp perform bettter at the olympics. 
2. How does hosting the olympic games, affect the economy GDP.

### What kind of stakeholders would be interested in analysis:

- Government & Policy Makers: To guide investment in sports and assess economic impacts.

- National Olympic Committees (NOCs): For strategic resource allocation and performance evaluation.

- Sports Federations: To advocate for funding based on performance data.

- Economic Analysts: To assess the economic impact of hosting the Olympics.

- IOC & Event Organizers: To evaluate benefits of hosting and improve future bids.

- Sponsors & Investors: For data-driven sponsorship and marketing decisions.

- Media & Journalists: To provide insights and compelling stories to the public.

- Academics & Researchers: To explore the relationship between economy and sports performance.


![title](image.png)


# Starnet and Granuality of the datawarehouse

To make the analysis possible we have to create a datawarehouse schema and starnet that will form the basis for our dimensions and our queries.

for this dataset we aim to have 5 dimension tables and 1 fact key ordered as follows:

### Data Warehouse Schema

The following tables will be created in the data warehouse to organize and store the data for Olympic performance and economic indicators (GDP) for each country.

1. **dim_country Table**  
   Stores information about the countries.  
   | Column         | Data Type | Description                   |
   |----------------|-----------|-------------------------------|
   | `country_id`   | SERIAL    | Primary key                   |
   | `country_name` | TEXT      | Name of the country           |
   | `continent`    | TEXT      | Continent of the country      |

2. **dim_year Table**  
   Stores information about the years and the host city for each Olympics.  
   | Column       | Data Type | Description                    |
   |--------------|-----------|--------------------------------|
   | `year_id`    | SERIAL    | Primary key                    |
   | `year`       | INTEGER   | Year of the Olympics           |
   | `host_city`  | TEXT      | Host city of the event         |

3. **dim_event Table**  
   Stores details about the sports and events.  
   | Column     | Data Type | Description                         |
   |------------|-----------|-------------------------------------|
   | `event_id` | SERIAL    | Primary key                         |
   | `event_name` | TEXT    | Name of the event                   |
   | `sport`    | TEXT      | Sport category (e.g., Athletics)    |

4. **dim_medal Table** (Optional but useful)  
   Stores the types of medals awarded.  
   | Column      | Data Type | Description               |
   |-------------|-----------|---------------------------|
   | `medal_id`  | SERIAL    | Primary key               |
   | `medal_type`| TEXT      | Medal type (Gold, Silver, Bronze) |

5. **dim_gdp Table**  
   Stores GDP data for each country and year.  
   | Column      | Data Type | Description               |
   |-------------|-----------|---------------------------|
   | `gdp_id`    | SERIAL    | Primary key               |
   | `country_id`| INTEGER   | Foreign key to `dim_country` |
   | `year_id`   | INTEGER   | Foreign key to `dim_year`  |
   | `gdp`       | NUMERIC   | GDP value for the country  |

6. **fact_medal_counts Table**  
   Stores the main measures, including medal counts and GDP.  
   | Column        | Data Type | Description                  |
   |---------------|-----------|------------------------------|
   | `fact_id`     | SERIAL    | Primary key                  |
   | `country_id`  | INTEGER   | Foreign key to `dim_country`  |
   | `year_id`     | INTEGER   | Foreign key to `dim_year`     |
   | `event_id`    | INTEGER   | Foreign key to `dim_event`    |
   | `gold_count`  | INTEGER   | Number of Gold medals        |
   | `silver_count`| INTEGER   | Number of Silver medals      |
   | `bronze_count`| INTEGER   | Number of Bronze medals      |
   | `total_medals`| INTEGER   | Total medals won             |
   | `gdp`         | NUMERIC   | GDP value for the year       |

# ETL Process 
Below we conduct the etl, process creating our different dimensions and then fact tables. once we do that we will then connect to a pgadmin remost server and from there build hierachies and so forth.

## Extraction

Since the data was just simplem, csv downlaod the extraction process will just be a series of code where we just load the data into the our staging area the jupyter notebook.


In [74]:
athlete_medal_data_raw = "AthleteMedalData.csv"
gdp_data_raw = "gdp data.csv"

## Transformation

In [75]:
import csv

def open_medal_athelte_data(data):
    """ This function will open the original athlete
        and medal raw CSV into our staging area, and then
        convert it into a Python pandas dataframe. """
    
    datalist = []
    with open(data, "rt") as raw_ath_med:
        reader = csv.reader(raw_ath_med)
        next(reader)  # Skip the header row
        
        for row in reader:
            # Ensuring we have the right number of columns
            if len(row) == 11:
                player_id, Name, Sex, Team, NOC, Year, Season, City, Sport, Event, Medal = row
                datalist.append((player_id, Name, Sex, Team, NOC, Year, Season, City, Sport, Event, Medal))
    
    return datalist

# Example usage
medal_athelte_data = open_medal_athelte_data(athlete_medal_data_raw)

**medal_data_df_and_formatting** `function`

In [ ]:

def medal_data_df_and_formatting(data):
    """ this function Converting data list into pandas dataframe and then 
        removes all rows with a "No medal" string value in the Medal column """

    # Converting data list into pandas dataframe

    athelete_medals_events_df = pd.DataFrame(data, columns = ["player_id", "Name", "Sex", "Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"])

    # Removing all rows with a "No medal" string value in the Medal column

    athelete_medals_events_df = athelete_medals_events_df[athelete_medals_events_df["Medal"] != "No medal"]

    return athelete_medals_events_df

In [82]:
athlete_medals_df = medal_data_df_and_formatting(open_medal_athelte_data(athlete_medal_data_raw))
athlete_medals_df.head()

,player_id,Name,Sex,Team,NOC,Year,Season,City,Sport,Event,Medal
3,3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
12,37,Arvo Aaltonen,M,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
13,38,Arvo Aaltonen,M,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
15,41,Paavo Aaltonen,M,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
16,42,Paavo Aaltonen,M,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold


Now that we have loaded in the data, let us investigate if there is any missig entries for anyof the fields.

In [83]:
athlete_medals_df.describe

<bound method NDFrame.describe of        player_id             Name Sex            Team  NOC  Year  Season  \
3              3      Edgar Aabye   M  Denmark/Sweden  DEN  1900  Summer   
12            37    Arvo Aaltonen   M         Finland  FIN  1920  Summer   
13            38    Arvo Aaltonen   M         Finland  FIN  1920  Summer   
15            41   Paavo Aaltonen   M         Finland  FIN  1948  Summer   
16            42   Paavo Aaltonen   M         Finland  FIN  1948  Summer   
...          ...              ...  ..             ...  ...   ...     ...   
252551   4979564    Quincy Wilson   M   United States  USA  2024  Summer   
252556   4980004         van Anne   F     Netherlands  NED  2024  Summer   
252562   1972077     Isayah Boers   M     Netherlands  NED  2024  Summer   
252563   1899865      Kevin Staut   M          France  FRA  2024  Summer   
252564   1924402  Charlie Carvell   M   Great Britain  GBR  2024  Summer   

             City       Sport                        

In [84]:
athlete_medals_df.isna().sum()

player_id    0
Name         0
Sex          0
Team         0
NOC          0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal        0
dtype: int64

For every element there is no missing entries.
We will move unto remove those columns where the, Medal type is "No medal" as our analysis will not be focused on collective particpation rates rather on the performance of those countries that particpated. 

**year_df_indexing** `function`

This function creates a subset of year and city from the athlete_medals_df, 
naming it year_df and it also resets the index adding a unique identifier year_id

In [88]:
def year_df_indexing(dataframe):
    """ This function creates a subset of year and city from the athlete_medals_df, 
        naming it year_df and it also resets the index adding a unique identifier year_id"""

    year_df  = dataframe[["Year","City"]]
    year_df.drop_duplicates(subset=["Year", "City"], keep="first", inplace=True)

    # adding a unique identifier as an index to the newly made dataframe

    year_df = year_df.reset_index()
    year_df["Year_id"] = year_df.index
    year_df = year_df.drop(columns=["index"])
    year_df = year_df[["Year_id","Year","City"]]
    

    return year_df 



In [89]:
year_dataframe = year_df_indexing(athlete_medals_df)
year_dataframe

C:\Users\Maria\AppData\Local\Temp\ipykernel_3680\1243901530.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df.drop_duplicates(subset=["Year", "City"], keep="first", inplace=True)


,Year_id,Year,City
0,0,1900,Paris
1,1,1920,Antwerpen
2,2,1948,London
3,3,1952,Helsinki
4,4,2008,Beijing
5,5,1988,Seoul
6,6,1996,Atlanta
7,7,1912,Stockholm
8,8,1956,Melbourne
9,9,2016,Rio de Janeiro


Creating the year dataframe into a csv

In [95]:
year_dataframe.to_csv('dim_year.csv', index=False) 

The next part is to tranform the athelte medal data into an event dataframe

In [ ]:

def event_df_indexing(dataframe):

    """ This function creates a subset of Sport and Event  from the athlete_medals_df, 
            naming it year_df and it also resets the index adding a unique identifier year_id"""

    event_df = dataframe[["Sport","Event"]]   #creating a subset

    event_df.drop_duplicates(subset=["Sport","Event"], keep="first", inplace=True)

    event_df = event_df.reset_index()

    event_df["Event_id"] = event_df.index

    event_df = event_df.drop(columns=["index"])

    event_df = event_df[["Event_id", "Sport","Event"]]

    return event_df

In [93]:
event_df_dataframe = event_df_indexing(athlete_medals_df)
event_df_dataframe

C:\Users\Maria\AppData\Local\Temp\ipykernel_3680\2413774140.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df.drop_duplicates(subset=["Sport","Event"], keep="first", inplace=True)


,Event_id,Sport,Event
0,0,Tug-Of-War,Tug-Of-War Men's Tug-Of-War
1,1,Swimming,Swimming Men's 200 metres Breaststroke
2,2,Swimming,Swimming Men's 400 metres Breaststroke
3,3,Gymnastics,Gymnastics Men's Individual All-Around
4,4,Gymnastics,Gymnastics Men's Team All-Around
...,...,...,...
1086,1086,Swimming,Mixed 4 x 100m Medley Relay
1087,1087,Swimming,Women's 4 x 100m Medley Relay
1088,1088,Swimming,Women's 4 x 200m Freestyle Relay
1089,1089,Water Polo,Women


Now let us create the event dimension csv.

In [96]:
event_df_dataframe.to_csv("dim_event.csv", index= False)

### Opening and cleaning the gdp data 

In [ ]:
def opening_gdp_raw(data):
    """ this function will open the data for the gdp here we will 
        define the rows we need only from the years 2000 and up as
        that is what we will basinng our analysis upon."""
    gdp_list = []

    with open(data,"rt") as gdp_raw:
        gdp_raw_csv_read = csv.reader(gdp_raw, delimiter= ",")

        for line in range(5):         # skipping the first 5 rows due them containing metdata dat to the csv.
            next(gdp_raw_csv_read) 
        
        for column in gdp_raw_csv_read:
            Country_Name = column[0]
            Country_Code = column[1]
            year_2000 = column[44]
            year_2001 = column[45]
            year_2002 = column[46]
            year_2003 = column[47]
            year_2004 = column[48]
            year_2005 = column[49]
            year_2006 = column[50]
            year_2007 = column[51]
            year_2008 = column[52]
            year_2009 = column[53]
            year_2010 = column[54]
            year_2011 = column[55]
            year_2012 = column[56]
            year_2013 = column[57]
            year_2014 = column[58]
            year_2015 = column[59]
            year_2016 = column[60]
            year_2017 = column[61]
            year_2018 = column[62]
            year_2019 = column[63]
            year_2020 = column[64]
            year_2021 = column[65]
            year_2022 = column[66]
            year_2023 = column[67]
            
        

            # Append the relevant columns to the list as a tuple
            x.append((
                Country_Name,
                Country_Code,
                year_2000, year_2001, year_2002, year_2003, year_2004, year_2005,
                year_2006, year_2007, year_2008, year_2009, year_2010, year_2011,
                year_2012, year_2013, year_2014, year_2015, year_2016, year_2017,
                year_2018, year_2019, year_2020, year_2021, year_2022, year_2023
            ))

    
    return gdp_list  
gdp_raw_list =  opening_gdp_raw(gdp_data_raw)

    

Now that we have created out list of gdps for the years 2000 - 2023, the next step is to convert the data into a pandas dataframe.


**gdp_raw_list_to_df** `function`

In [ ]:
def gdp_raw_list_to_df(data):
    
    """ This function converst the gdp raw data list into a pandas's dataframe """


    gdp_raw_df = pd.DataFrame(data, columns= ['Country_Name',
    'Country_Code',
    'year_2000', 'year_2001', 'year_2002', 'year_2003', 'year_2004', 'year_2005',
    'year_2006', 'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011',
    'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017',
    'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023'])

    return gdp_raw_df

In [ ]:

gdp_raw_dataframe = gdp_raw_list_to_df(opening_gdp_raw(gdp_data_raw))
gdp_raw_dataframe.head()

,Country_Name,Country_Code,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,Aruba,ABW,21026.16709,20911.19277,21375.26912,22050.58987,24105.52412,24978.22167,25832.92556,27665.29948,29011.63906,25740.77023,24452.58874,26043.15633,25611.17577,26514.86898,26940.26411,28419.26453,28449.71295,29329.08175,30918.48358,31902.80982,24008.12782,29127.75938,33300.83882,
1,Africa Eastern and Southern,AFE,715.1425073,633.4733816,633.5202007,819.986705,994.1938544,1130.168938,1235.847125,1379.746756,1439.240924,1404.535948,1622.684093,1757.998404,1724.204053,1696.35604,1678.55361,1498.805084,1346.301281,1485.753579,1558.612079,1508.486886,1356.088871,1545.956697,1642.432039,1672.505957
2,Afghanistan,AFG,180.1883694,142.9033644,182.1740382,199.6432265,221.8305307,254.115276,274.0153923,376.3182996,382.5338072,453.3873851,562.4992216,608.7388504,653.4174749,638.733181,626.5129291,566.8811297,523.053012,526.140801,492.090631,497.7414313,512.055098,355.7778264,352.6037331,
3,Africa Western and Central,AFW,527.2025617,539.2257488,627.0959153,706.4765613,850.0835822,1008.341766,1246.349454,1421.444644,1685.806936,1467.789174,1680.141063,1861.537303,1958.077403,2154.315084,2248.518426,1882.518808,1648.920269,1590.555785,1735.445833,1813.609146,1688.470871,1769.171853,1788.875347,1584.333285
4,Angola,AGO,556.884253,527.4641105,872.6576314,982.8055874,1254.696119,1900.723809,2597.963587,3121.348725,4081.717506,3123.698885,3586.663694,4608.155166,5083.826873,5061.349253,5011.984427,3217.33924,1809.709377,2439.374439,2540.508879,2191.347764,1450.905111,1927.474078,2933.484644,2309.52162


The data here is longitdal, let us change it into a wide format using the python function melt.

In [101]:
def gdp_raw_to_wideformat(dataframe):  

    # Defining the

    gdp_raw_df_wide = pd.melt(dataframe ,id_vars=["Country_Name", "Country_Code"], 
                        var_name= "Year", value_name= "Value")
    
    # we then also want to remove the year prefix..

    gdp_raw_df_wide['Year'] = gdp_raw_df_wide['Year'].str.replace('year_', '')

    return gdp_raw_df_wide


<!-- Next let us check if there is missing data, we know gdp and ecnomic indicators could be sensitive infromation to some governments and they may wish to not, disclose their gdp, as well as other circumstances such as country economic state and organisation not permitting them to accurtaltely report their data. 

An exeprt from the world bank where this data was retrieved from states: etc

from this analysis we will try a range of methods to try and see if we can impute values into this missing data. -->

In [103]:
gdp_wide = gdp_raw_to_wideformat(gdp_raw_list_to_df(opening_gdp_raw(gdp_data_raw)))
gdp_wide

,Country_Name,Country_Code,Year,Value
0,Aruba,ABW,2000,21026.16709
1,Africa Eastern and Southern,AFE,2000,715.1425073
2,Afghanistan,AFG,2000,180.1883694
3,Africa Western and Central,AFW,2000,527.2025617
4,Angola,AGO,2000,556.884253
...,...,...,...,...
6379,Kosovo,XKX,2023,5943.125714
6380,"Yemen, Rep.",YEM,2023,
6381,South Africa,ZAF,2023,6253.161613
6382,Zambia,ZMB,2023,1369.129365


Some EDA, mainly of to search for missing gdp data

In [108]:
gdp_wide[gdp_wide['Value'] == ''].count()

Country_Name    244
Country_Code    244
Year            244
Value           244
dtype: int64

From we can tell that there are 244 rows in the dataset with, blank entries.
Let us see which countries have the most blank entries and then we will explore appropriate iputation methods.

In [113]:
gdp_wide[gdp_wide['Value'] == ''].groupby("Country_Name", observed =  False)["Value"].count()

Country_Name
Afghanistan                   1
American Samoa                3
Aruba                         1
Bermuda                       1
Bhutan                        1
British Virgin Islands       24
Cayman Islands                7
Channel Islands               9
Cuba                          3
Curacao                      12
Eritrea                      12
Faroe Islands                 1
French Polynesia              1
Gibraltar                    24
Greenland                     2
Guam                          3
Isle of Man                   2
Korea, Dem. People's Rep.    24
Kosovo                        8
Lebanon                       1
Liechtenstein                 1
Monaco                        1
New Caledonia                 1
Northern Mariana Islands      5
Not classified               24
Qatar                         1
San Marino                    2
Sint Maarten (Dutch part)    11
South Sudan                  16
St. Martin (French part)     20
Syrian Arab Republic       

In [120]:
gdp_wide[gdp_wide['Country_Name'] == 'Gibraltar']

,Country_Name,Country_Code,Year,Value
84,Gibraltar,GIB,2000,
350,Gibraltar,GIB,2001,
616,Gibraltar,GIB,2002,
882,Gibraltar,GIB,2003,
1148,Gibraltar,GIB,2004,
1414,Gibraltar,GIB,2005,
1680,Gibraltar,GIB,2006,
1946,Gibraltar,GIB,2007,
2212,Gibraltar,GIB,2008,
2478,Gibraltar,GIB,2009,


**The hard part**
